In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589283290102_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = sqlContext.read.csv('s3a://sparkdemonstration/10M.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
catCols = ['C1'] + [f'C{k}' for k in range(14, 22)]
catCols += ['banner_pos', 'site_category', 'device_type', 'device_conn_type',
            'site_id', 'site_domain', 'app_id', 'app_domain', 'app_category', 'device_model']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.functions import udf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
posMapper = udf(lambda x: 0 if x < 0 else x)
df = df.withColumn('C20_1', posMapper(df['C20']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
catCols.remove('C20')
catCols.append('C20_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.ml.feature import OneHotEncoderEstimator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.ml.feature import StringIndexer
for c in ['site_category', 'site_id', 'site_domain', 'app_id', 'app_domain', 'app_category', 'device_model']:
    si = StringIndexer(inputCol=c, outputCol=f'{c}_ix')
    df = si.fit(df).transform(df)
    catCols.remove(c)
    catCols.append(f'{c}_ix')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql.types import IntegerType
df = df.withColumn("C20_1int", df['C20_1'].cast(IntegerType()))
catCols.remove('C20_1')
catCols.append('C20_1int')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
encoder = OneHotEncoderEstimator(inputCols=catCols, outputCols=[c + 'Enc' for c in catCols])
enc_model = encoder.fit(df)
encoded = enc_model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
trainCols = [c for c in encoded.columns if c.endswith('Enc')] + ['day', 'hour', 'dayofweek']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
encoded = encoded.withColumn('label', encoded['click'].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
va = VectorAssembler(inputCols=trainCols, outputCol='features')
encoded = va.transform(encoded)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
auc = []
recall = []
L1 = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
for l in L1:
    lr = LogisticRegression(featuresCol='features', labelCol='label', elasticNetParam=1, regParam=l)
    model = lr.fit(encoded)
    result = model.evaluate(encoded)
    auc.append(result.areaUnderROC)
    r = result.recallByLabel
    recall.append(r)
    print(l, r)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://172.31.86.191:18888/sessions/1/statements/16 with error payload: {"msg":"requirement failed: Session isn't active."}


In [17]:
spark.sparkContext.getConf().get('spark.driver.memory')

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.86.191:18888/sessions/1 with error payload: {"msg":"Session '1' not found."}


In [21]:
auc

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.86.191:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [22]:
recall

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.86.191:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [29]:
auc = []
recall = []
L1 = [0.0001, 0.0002, 0.0005, 0.0007, 0.001]
for l in tqdm(L1):
    lr = LogisticRegression(featuresCol='features', labelCol='label', elasticNetParam=1, regParam=l)
    model = lr.fit(encoded)
    result = model.evaluate(encoded)
    auc.append(result.areaUnderROC)
    recall.append(result.recallByLabel)

100%|██████████| 5/5 [18:31<00:00, 225.08s/it]


In [18]:
auc

[0.7617144797424745,
 0.7616922664507153,
 0.7612807183694248,
 0.7602531474308007,
 0.7577812378170214]

In [19]:
recall

[[0.9881304851637087, 0.09845999823326756],
 [0.9882099846541893, 0.09809487353140367],
 [0.9882665979277133, 0.09754718647860781],
 [0.9887243222668437, 0.09458496510703454],
 [0.9891290469456536, 0.09082182503459851]]

In [17]:
auc = []
recall = []
L1 = [0.00001, 0.00002, 0.00005, 0.0001, 0.0002]
for l in tqdm(L1):
    lr = LogisticRegression(featuresCol='features', labelCol='label', elasticNetParam=1, regParam=l)
    model = lr.fit(encoded)
    result = model.evaluate(encoded)
    auc.append(result.areaUnderROC)
    recall.append(result.recallByLabel)

100%|██████████| 5/5 [18:33<00:00, 219.25s/it]


In [20]:
result.precisionByLabel

[0.8417480564849439, 0.630834049167587]

In [21]:
auc

[0.7617144797424745,
 0.7616922664507153,
 0.7612807183694248,
 0.7602531474308007,
 0.7577812378170214]

In [22]:
recall

[[0.9881304851637087, 0.09845999823326756],
 [0.9882099846541893, 0.09809487353140367],
 [0.9882665979277133, 0.09754718647860781],
 [0.9887243222668437, 0.09458496510703454],
 [0.9891290469456536, 0.09082182503459851]]

In [28]:
auc = []
recall = []
L1 = [0.000001, 0.000002, 0.000005, 0.00001]
for l in tqdm(L1):
    lr = LogisticRegression(featuresCol='features', labelCol='label', elasticNetParam=1, regParam=l)
    model = lr.fit(encoded)
    result = model.evaluate(encoded)
    auc.append(result.areaUnderROC)
    recall.append(result.recallByLabel)

100%|██████████| 4/4 [12:39<00:00, 191.37s/it]


In [29]:
auc

[0.7617552201376266,
 0.7617331959413733,
 0.7617276905715271,
 0.7617163913010492]

In [30]:
recall

[[0.9879775088714204, 0.09903713082653633],
 [0.9880003950883769, 0.09906068725891463],
 [0.988013645003457, 0.09891934866464475],
 [0.9881304851637087, 0.09845999823326756]]

In [33]:
auc = []
recall = []
L1 = [0.0000001, 0.0000002, 0.0000005, 0.000001]
for l in tqdm(L1):
    lr = LogisticRegression(featuresCol='features', labelCol='label', elasticNetParam=1, regParam=l)
    model = lr.fit(encoded)
    result = model.evaluate(encoded)
    auc.append(result.areaUnderROC)
    recall.append(result.recallByLabel)


100%|██████████| 4/4 [14:39<00:00, 227.52s/it]

In [34]:
auc

[0.7617689728508258,
 0.7617775921073082,
 0.7617591900401888,
 0.7617539369029065]

In [35]:
recall

[[0.987811282664052, 0.09985571685168282],
 [0.9877956236735028, 0.0999734990135744],
 [0.9879642589563403, 0.0992668060422249],
 [0.9879775088714204, 0.09903713082653633]]